In [1]:
import pandas as pd
import numpy as np
import IPython.display as ipd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import sklearn.preprocessing, sklearn.decomposition, sklearn.svm
from sklearn.model_selection import train_test_split
import sklearn.ensemble
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score

### EDA

In [2]:
tracks = pd.read_csv('fma_metadata/tracks.csv',index_col=0,header=[0,1])
#genres = pd.read_csv('data/fma_metadata/genres.csv')
features = pd.read_csv('fma_metadata/features.csv',index_col=0,header=[0,1,2])
#echonest = pd.read_csv('data/fma_metadata/echonest.csv')

In [3]:
ipd.display(tracks['track'].head())
#ipd.display(tracks['album'].head())
#ipd.display(tracks['artist'].head())
#ipd.display(tracks['set'].head())

,bit_rate,comments,composer,date_created,date_recorded,duration,favorites,genre_top,genres,genres_all,information,interest,language_code,license,listens,lyricist,number,publisher,tags,title
track_id,,,,,,,,,,,,,,,,,,,,
2,256000,0,NaN,2008-11-26 01:48:12,2008-11-26 00:00:00,168,2,Hip-Hop,[21],[21],NaN,4656,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1293,NaN,3,NaN,[],Food
3,256000,0,NaN,2008-11-26 01:48:14,2008-11-26 00:00:00,237,1,Hip-Hop,[21],[21],NaN,1470,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,514,NaN,4,NaN,[],Electric Ave
5,256000,0,NaN,2008-11-26 01:48:20,2008-11-26 00:00:00,206,6,Hip-Hop,[21],[21],NaN,1933,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1151,NaN,6,NaN,[],This World
10,192000,0,Kurt Vile,2008-11-25 17:49:06,2008-11-26 00:00:00,161,178,Pop,[10],[10],NaN,54881,en,Attribution-NonCommercial-NoDerivatives (aka M...,50135,NaN,1,NaN,[],Freeway
20,256000,0,NaN,2008-11-26 01:48:56,2008-01-01 00:00:00,311,0,NaN,"[76, 103]","[17, 10, 76, 103]",NaN,978,en,Attribution-NonCommercial-NoDerivatives (aka M...,361,NaN,3,NaN,[],Spiritual Level


In [4]:
columns = ['mfcc', 'chroma_cens', 'tonnetz', 'spectral_contrast','spectral_rolloff','zcr','rmse']
ipd.display(features[columns].head().style.format('{:.2f}'))

In [6]:
features.shape

(106574, 518)

In [7]:
genres = pd.DataFrame(tracks['track']['genre_top'])
genres['genre_top'].value_counts()

Rock                   14182
Experimental           10608
Electronic              9372
Hip-Hop                 3552
Folk                    2803
Pop                     2332
Instrumental            2079
International           1389
Classical               1230
Jazz                     571
Old-Time / Historic      554
Spoken                   423
Country                  194
Soul-RnB                 175
Blues                    110
Easy Listening            24
Name: genre_top, dtype: int64

In [9]:
plt.figure(figsize=(12,6))
sns.boxplot
features.boxplot(by ='day', column =['total_bill'], grid = False)

KeyError: 'day'

<Figure size 1200x600 with 0 Axes>

### Preprocessing

In [8]:
# columns = ['chroma_cens', 'tonnetz', 'spectral_contrast','spectral_rolloff','zcr','rmse','chroma_stft']
# combined_features = features['mfcc'][['mean','std']]
# for column in columns:
#     combined_features = pd.concat((combined_features, features[column][['mean','std']]), axis=1)
combined_features = features['mfcc']
combined_featuresa

statistics   kurtosis                                                    \
number             01        02        03        04        05        06   
track_id                                                                  
2            3.856789  1.541901  0.000816  0.330728  0.118731 -0.342687   
3            4.296755  1.399977  0.112535 -0.211170  0.032953 -0.023489   
5            2.624517  2.415293  0.440233 -0.782131 -0.771069 -0.724216   
10           5.076893  1.161854  2.095651  1.372743 -0.203574 -0.345354   
20          11.880132  4.085320  0.002655  1.518562  0.181947  0.339014   
...               ...       ...       ...       ...       ...       ...   
155316       4.895152  5.361988  3.186483  0.510951 -0.143080  0.342237   
155317       0.040857  1.752112  0.442188 -0.380960 -0.739809 -0.023362   
155318       0.581889 -0.129386  0.662489  1.025676  0.170532  1.827692   
155319       5.164501  3.716359  2.881116  3.273014  0.707058  1.951918   
155320       4.619462  1.234909  1.236056  0.335577  0.009484  1.682914   

statistics                                          ...        std             \
number            07        08        09        10  ...         11         12   
track_id                                            ...                         
2          -0.259252  0.146735  0.410656 -0.162872  ...  10.059609   8.601942   
3           0.150404  0.046454  0.033484 -0.064596  ...   8.771966  10.026867   
5           0.090260  0.152119  0.261731 -0.608905  ...   8.863638   9.581952   
10         -0.529139  0.561974  0.281350 -0.150672  ...   8.289734   7.985110   
20          0.367611  0.069932 -0.016032  0.026012  ...   7.706648   6.498904   
...              ...       ...       ...       ...  ...        ...        ...   
155316      0.771449 -0.184232  0.143237  0.009016  ...   7.401937   6.629143   
155317      0.264786  0.072084  0.295485  0.535544  ...   6.734644   6.955739   
155318     -0.189439  1.141878  0.022045  0.175235  ...   7.061307   6.943147   
155319     -0.151555  0.230036 -0.327132  1.182421  ...   7.300332   5.971618   
155320      0.317069 -0.128262  0.076033  0.112258  ...   7.538454   7.840639   

statistics                                                              \
number            13        14        15        16        17        18   
track_id                                                                 
2           9.284250  9.245516  8.520863  8.560472  7.651871  7.246555   
3           6.978541  7.650417  9.600357  7.222888  8.398293  7.285423   
5           8.895723  8.141456  8.201844  7.780963  7.132692  7.539753   
10          7.075400  6.972649  7.071393  7.270959  7.051070  6.928591   
20          6.845514  6.934421  7.019398  6.983841  6.813648  7.520811   
...              ...       ...       ...       ...       ...       ...   
155316      5.860062  6.230947  5.923375  5.586200  5.516872  5.755764   
155317      6.662989  6.478699  6.019901  5.757353  5.280114  5.674062   
155318      6.522048  6.780707  6.311563  5.789986  5.466107  5.607614   
155319      6.355647  5.602213  5.887583  5.110033  5.407447  5.317993   
155320      6.794503  6.431658  6.400701  6.733516  5.603289  5.868379   

statistics                      
number            19        20  
track_id                        
2           7.077188  7.391859  
3           7.417791  8.777440  
5           8.452527  7.334442  
10          6.430473  6.186294  
20          7.098001  7.032246  
...              ...       ...  
155316      5.105072  5.019556  
155317      5.541949  5.266131  
155318      5.685448  5.449005  
155319      5.187994  5.260817  
155320      6.128483  5.550875  

[106574 rows x 140 columns]

In [32]:
grouped_genres = genres.groupby('genre_top')
balanced_genres = pd.DataFrame(columns=['genre_top'])
genre_list = ['Rock','Experimental','Electronic','Hip-Hop','Folk','Pop','Instrumental']
#ipd.display(grouped_genres.get_group('Rock')[:2000].reset_index())
for genre in genre_list:
    balanced_genres = pd.concat([balanced_genres,grouped_genres.get_group(genre)[:3000]])

print(balanced_genres['genre_top'].value_counts())
balanced_genres

Rock            3000
Experimental    3000
Electronic      3000
Hip-Hop         3000
Folk            2803
Pop             2332
Instrumental    2079
Name: genre_top, dtype: int64


,genre_top
135,Rock
136,Rock
151,Rock
152,Rock
153,Rock
...,...
155293,Instrumental
155294,Instrumental
155295,Instrumental
155296,Instrumental


In [33]:
track_ids = balanced_genres.index.tolist()
track_features = combined_features.loc[track_ids]
track_features

statistics   kurtosis                                                     \
number             01         02        03        04        05        06   
track_id                                                                   
135          2.021638   0.269172  0.755836  0.861484  1.909718  0.382428   
136         13.691644   0.702855  2.484976  7.980183  0.209275  1.472870   
151          8.044993   1.115101  1.708385  0.482039  0.088183  0.000533   
152          7.162528  -0.251133  1.689257  0.128104 -0.853159 -0.233372   
153         14.468616   3.223378 -0.510767 -0.652397 -0.419132  0.366079   
...               ...        ...       ...       ...       ...       ...   
155293      -1.210663   3.031699 -1.433665 -0.409859 -0.607855 -0.349818   
155294      -0.274696  -0.053879 -0.147241  0.384711 -0.532430  0.608346   
155295      -0.850595  10.226659 -1.620962 -0.842667  2.726631  1.993919   
155296      -0.238809  -0.043569 -0.013263 -0.974981  0.274343 -0.792175   
155297      -0.248032  12.852261 -0.847872 -0.570691 -0.404573 -0.194173   

statistics                                          ...        std             \
number            07        08        09        10  ...         11         12   
track_id                                            ...                         
135         3.033167  0.276814  0.344835  0.095461  ...   8.329439   8.129856   
136         2.104760  0.164231  0.288295  0.212297  ...   6.960906   8.477464   
151         0.008176  0.335411 -0.134841  0.068216  ...   7.174704   6.137657   
152         0.042815  1.103065 -0.759770 -0.679819  ...   8.083424   6.650053   
153        -0.226173  0.183614  0.216913  0.083776  ...   6.114115   6.469112   
...              ...       ...       ...       ...  ...        ...        ...   
155293      0.949396 -0.639051  0.156564 -0.370559  ...   7.540020   7.943772   
155294      0.694465  0.411823  0.339724  0.312192  ...   6.873684   6.430038   
155295      1.363848  3.109032  0.087855  0.481166  ...   8.226966   7.802309   
155296     -0.417961 -0.646503 -0.453943 -0.497149  ...  12.253113  13.038336   
155297     -0.374795 -0.212409 -0.080454  0.221916  ...   4.958858   4.952670   

statistics                                                               \
number            13         14        15        16        17        18   
track_id                                                                  
135         7.796185   6.755871  6.407472  5.984369  6.050004  5.692363   
136         8.295549   8.519705  8.484787  7.838695  7.337792  7.240948   
151         6.587673   6.327073  8.807613  7.392186  6.808066  6.320422   
152         6.340897   6.097323  6.395099  6.092227  5.699416  5.911937   
153         4.984094   5.218048  4.981048  6.233972  4.723229  6.117352   
...              ...        ...       ...       ...       ...       ...   
155293      6.586681   6.833907  6.883475  6.481982  5.497596  5.441007   
155294      6.640862   7.800838  7.836264  6.585768  7.932791  7.807853   
155295      7.124853   7.206721  7.930612  7.349055  7.508310  6.937418   
155296      9.914006  10.874232  7.665954  7.383183  8.016631  9.450324   
155297      5.350151   5.567447  6.213673  5.275000  5.329784  4.990051   

statistics                      
number            19        20  
track_id                        
135         6.055962  5.925325  
136         6.968402  6.127878  
151         6.074442  6.507479  
152         6.624181  5.551071  
153         4.750901  5.295139  
...              ...       ...  
155293      5.242710  5.268094  
155294      7.733168  8.541412  
155295      5.994994  6.674896  
155296      7.297700  8.140812  
155297      5.008690  5.432521  

[19214 rows x 140 columns]

In [64]:
data = pd.concat([track_features,balanced_genres],axis=1)
#data.boxplot(by ='genre_top', column = [], grid = False)
data

,"(kurtosis, 01)","(kurtosis, 02)","(kurtosis, 03)","(kurtosis, 04)","(kurtosis, 05)","(kurtosis, 06)","(kurtosis, 07)","(kurtosis, 08)","(kurtosis, 09)","(kurtosis, 10)",...,"(std, 12)","(std, 13)","(std, 14)","(std, 15)","(std, 16)","(std, 17)","(std, 18)","(std, 19)","(std, 20)",genre_top
135,2.021638,0.269172,0.755836,0.861484,1.909718,0.382428,3.033167,0.276814,0.344835,0.095461,...,8.129856,7.796185,6.755871,6.407472,5.984369,6.050004,5.692363,6.055962,5.925325,Rock
136,13.691644,0.702855,2.484976,7.980183,0.209275,1.472870,2.104760,0.164231,0.288295,0.212297,...,8.477464,8.295549,8.519705,8.484787,7.838695,7.337792,7.240948,6.968402,6.127878,Rock
151,8.044993,1.115101,1.708385,0.482039,0.088183,0.000533,0.008176,0.335411,-0.134841,0.068216,...,6.137657,6.587673,6.327073,8.807613,7.392186,6.808066,6.320422,6.074442,6.507479,Rock
152,7.162528,-0.251133,1.689257,0.128104,-0.853159,-0.233372,0.042815,1.103065,-0.759770,-0.679819,...,6.650053,6.340897,6.097323,6.395099,6.092227,5.699416,5.911937,6.624181,5.551071,Rock
153,14.468616,3.223378,-0.510767,-0.652397,-0.419132,0.366079,-0.226173,0.183614,0.216913,0.083776,...,6.469112,4.984094,5.218048,4.981048,6.233972,4.723229,6.117352,4.750901,5.295139,Rock
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155293,-1.210663,3.031699,-1.433665,-0.409859,-0.607855,-0.349818,0.949396,-0.639051,0.156564,-0.370559,...,7.943772,6.586681,6.833907,6.883475,6.481982,5.497596,5.441007,5.242710,5.268094,Instrumental
155294,-0.274696,-0.053879,-0.147241,0.384711,-0.532430,0.608346,0.694465,0.411823,0.339724,0.312192,...,6.430038,6.640862,7.800838,7.836264,6.585768,7.932791,7.807853,7.733168,8.541412,Instrumental
155295,-0.850595,10.226659,-1.620962,-0.842667,2.726631,1.993919,1.363848,3.109032,0.087855,0.481166,...,7.802309,7.124853,7.206721,7.930612,7.349055,7.508310,6.937418,5.994994,6.674896,Instrumental
155296,-0.238809,-0.043569,-0.013263,-0.974981,0.274343,-0.792175,-0.417961,-0.646503,-0.453943,-0.497149,...,13.038336,9.914006,10.874232,7.665954,7.383183,8.016631,9.450324,7.297700,8.140812,Instrumental


In [34]:
x,y = sklearn.utils.shuffle(track_features,balanced_genres['genre_top'].to_frame(),random_state=42)

In [35]:
ipd.display(x.head(20))
ipd.display(y.head(20))

statistics   kurtosis                                                     \
number             01         02        03        04        05        06   
track_id                                                                   
13192       -0.458224  -0.568430 -0.453206  0.831239  0.947356  0.161656   
12597        2.328145  -0.713074 -0.867640 -0.183959  1.072172  1.420972   
124175       4.842550  11.361656  0.220301 -0.138751 -0.035101 -0.206867   
3264        -0.791766   5.118561 -0.885679  4.372694  0.969028  0.985327   
7557         0.714817   2.167044 -1.147677 -0.771478  2.783834 -0.697526   
9090        -0.458637   1.121837  0.502786 -0.744306 -0.924978  0.270297   
165         10.204412   2.827944  4.927337  2.089111  1.859405  0.145997   
151846      -0.426428  -0.244018  0.875004 -0.487099 -0.427132 -0.168090   
13573       -0.402205  -0.600808 -1.050359 -0.327304  0.243953 -0.219979   
34507        4.649978   1.297379 -0.084364  0.982255 -0.169523 -0.110644   
5004         0.418569  -0.435099 -0.317104  0.575605  0.038442  0.779131   
108429       0.848228   6.251531  0.588347 -0.283532  0.569665 -0.194644   
119585       0.323389   4.602067 -0.691581  1.833214  1.051090  0.781786   
7348        -0.186890  -0.266707 -0.095696 -0.468169  0.010090 -0.488088   
148288       8.158388  13.088209  1.522878 -0.177451  0.142449  0.344740   
27465        2.126522   0.213576  0.973462  0.872049  0.338470  0.900343   
129691       4.195949   3.797563  1.001099  0.470660  0.431865 -0.018055   
144545       1.726051   2.599703  0.310971  1.147010  0.415228  0.214296   
115303       4.740364   1.364091 -0.411607  0.566738 -0.096701 -0.233133   
29197        0.614578   4.504858 -0.325606  0.405142 -0.101286 -0.073379   

statistics                                          ...        std             \
number            07        08        09        10  ...         11         12   
track_id                                            ...                         
13192      -0.192054  0.051341 -0.085989  0.532652  ...   8.666471   9.476106   
12597       0.456861  2.401669  0.708404  0.180036  ...   8.426355   8.057528   
124175      0.011808  0.146207  0.151088 -0.250708  ...   7.982253   6.918171   
3264        0.236096 -0.462945  1.026338 -0.105461  ...  10.818964  10.117365   
7557       -0.199300  0.205305 -0.276903  0.091899  ...  11.065008   9.177638   
9090       -0.804997 -0.363206 -0.892862 -0.630322  ...   9.673862   8.885182   
165         1.786952  0.082265 -0.167573 -0.062014  ...   7.015270   7.117929   
151846     -0.171795  0.058597 -0.490142  0.021948  ...  10.446978  13.546087   
13573      -0.324575 -0.119134  0.004474  1.298408  ...   9.187920   8.447997   
34507       0.936369  0.080838  0.338480  1.671780  ...   6.727410   6.741544   
5004       -0.142895 -0.301418 -0.392971  0.284055  ...  10.453803  10.121677   
108429      0.660864  0.239976  0.327592  0.159349  ...   8.481928   7.515728   
119585      0.313291  0.004038  0.604322  0.363867  ...   9.993578   8.810262   
7348        0.427538 -0.263035  0.150153  0.252679  ...  10.016620  10.060058   
148288      0.512887  0.028314  0.014612 -0.152801  ...   7.354143   7.406994   
27465       0.508534  0.699074  0.554149  0.641422  ...   9.645339  10.235584   
129691      0.577603  0.304432  0.449610  0.459827  ...   7.191748   7.974715   
144545     -0.364559  0.245684  0.099554  0.399813  ...   9.580902   8.771581   
115303      0.116936  0.166567 -0.277735  0.531886  ...   7.794047   5.532470   
29197      -0.040231 -0.098392  0.470309  0.677332  ...  15.510163  15.734292   

statistics                                                                    \
number             13         14         15         16         17         18   
track_id                                                                       
13192        9.170434   8.713119   8.560030   7.662582   7.190042   7.523359   
12597        7.458536   6.187005   5.873287   5.742706   5.546103  

,genre_top
13192,Electronic
12597,Rock
124175,Instrumental
3264,Folk
7557,Experimental
9090,Electronic
165,Experimental
151846,Folk
13573,Experimental
34507,Experimental


In [36]:
scaler = sklearn.preprocessing.MinMaxScaler()
x = scaler.fit_transform(x)

In [37]:
y['genre'] = pd.factorize(y['genre_top'])[0]
y

,genre_top,genre
13192,Electronic,0
12597,Rock,1
124175,Instrumental,2
3264,Folk,3
7557,Experimental,4
...,...,...
103797,Hip-Hop,6
125653,Hip-Hop,6
35010,Experimental,4
3761,Rock,1


In [38]:
x_train, x_test = train_test_split(x, test_size=0.2, random_state=1337)
y_train, y_test = train_test_split(y['genre'], test_size=0.2, random_state=1337)

In [49]:
def model_assess(model):
    model.fit(x_train, y_train)
    preds = model.predict(x_test)
    #print(confusion_matrix(y_test, preds))
    print('Test Accuracy', ':', round(accuracy_score(y_test, preds), 5), '\n')
    print('Train Accuracy', ':', round(model.score(x_train,y_train), 5), '\n')

In [50]:
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier, XGBRFClassifier

In [51]:
clf = sklearn.svm.SVC()
model_assess(clf)

Test Accuracy : 0.55425 

Train Accuracy : 0.57413 



In [52]:
rf = sklearn.ensemble.RandomForestClassifier()
model_assess(rf)

Test Accuracy : 0.55712 

Train Accuracy : 0.99857 



In [53]:
ada = AdaBoostClassifier(n_estimators=1000, random_state=0)
model_assess(ada)

Test Accuracy : 0.48972 

Train Accuracy : 0.51747 



In [54]:
knn = KNeighborsClassifier(n_neighbors=11)
model_assess(knn)

Test Accuracy : 0.53136 

Train Accuracy : 0.58396 



In [ ]:
xgb = XGBClassifier(n_estimators=500, learning_rate=0.02,colsample_bytree=0.5,colsample_bylevel=0.5,colsample_bynode=0.5,max_depth=5,gamma=2)
model_assess(xgb)